### This notebook can be used to pre-process the story content.

This notebook reuses code from the book "Practical Machine Learning with Python". The code is available in the follwoing repository (licensed unde Apache 2.0):

https://github.com/dipanjanS/practical-machine-learning-with-python

In [1]:
import pandas as pd
import unicodedata
import re
import spacy
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup

from contractions import CONTRACTION_MAP

pd.options.display.max_colwidth = 1000

/home/arnabb/anaconda3/envs/pb/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/arnabb/anaconda3/envs/pb/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/home/arnabb/anaconda3/envs/pb/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/arnabb/anaconda3/envs/pb/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [2]:
nlp = spacy.load('en', parse = True, tag=True, entity=True)

tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

/home/arnabb/anaconda3/envs/pb/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/arnabb/anaconda3/envs/pb/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [3]:
# Location of the stories file
ENGLISH_CSV_FILE = "/home/arnabb/vssexclude/d_s/PrathamBooks/PrathamBooks-Sprint-2018/data/gen_by_script/stories_content_english.csv"

In [4]:
df_english_stories = pd.read_csv(ENGLISH_CSV_FILE)

In [5]:
df_english_stories.columns

Index(['story_id', 'story_title', 'story_english_title',
       'is_child_created_story', 'stories_status', 'stories_summary',
       'ancestry', 'is_recommended_story', 'reads', 'language_name',
       'organization_name', 'page_type', 'story_derivation_type',
       'story_publishing_type', 'reading_level_cat', 'story_content'],
      dtype='object')

In [6]:
# Check the summary for each story
df_english_stories.iloc[0:5][['story_title', 'stories_summary']]

,story_title,stories_summary
0,Smile Please!,Follow the young deer as he races along with friends.
1,Fat King Thin Dog,Run along with the Fat King after the Thin Dog!
2,"Vayu, the Wind","A nice story about this silent and always present friend, the wind does a lot of things without anybody's notice!"
3,Chuskit Goes to School!,"Nine year old Chuskit longed to go to school, to make friends, learn math and play games. But she could not... until Abdul decided to do something about it."
4,"Not Now, Not Now!","The little boy in this story wants to do something now, but all his elders say NOT NOW! Do you get angry when someone says that to you?"


In [7]:
# Check the content of each story
df_english_stories.iloc[0:5][['story_title', 'story_content']]

,story_title,story_content
0,Smile Please!,"A fawn was racing in the forest. He was ahead of the rabbit. He was ahead of the elephant. He leapt and cleared the stream. He ran past the crumbling wall. There was a large boulder on the grassy plain. He stumbled and fell down. He burst into tears. The monkey massaged his leg. Tears flowed from the fawn's eyes. Brother Bear picked him up. The fawn didn't stop crying. His mother came. She said, “Look, we’ll beat up this bad boulder!” The fawn said, “Oh, don’t do that or he will also start crying.” His mother laughed. So did the fawn."
1,Fat King Thin Dog,This is a fat king. The fat king has a thin dog. The fat king and his thin dog go for a walk. The dog sees a bird. The dog runs after the bird. The king runs after the dog. They run and run. They run and run for many days. The king catches the dog. Now the fat king is thin.
2,"Vayu, the Wind","Everytime I finish my hot, hot bath, My wet body feels so cool, cool cool. Who makes that happen? Vayu the Wind! The milk in my cup is too hot, too hot. But soon it is ready for me to gulp. Who makes that happen? Vayu the Wind! The window curtains flutter And gently brush my face. Who makes it happen? Vayu, the Wind! A bolt of lightning far away. Black clouds moving my way. Who makes that happen? Vayu, the Wind! Branches sway and leaves tremble. Flowers gently fall. Who did it all? Vayu, the Wind! Far from the house, we are playing, Yet, I can smell the sweets mother is preparing. Who makes it happen? Vayu, the Wind! A glass tumbler on a window sill, Crashes on the ground. Thank God, I was not around. Who played this mischief? Of course, it was Vayu, the Wind! A whistle blows. A train rolls in. I cannot see, but I hear its din. Who makes that happen? Vayu, the Wind! Cannot be seen Cannot be heard Does all the work Without a word. Who can it be? Of course! The wind! Vayu, the wind!"
3,Chuskit Goes to School!,"Chuskit woke up early that morning. It was a very special day and she was too excited to sleep. She eagerly looked out of the room through the window next to her bed. It was spring time in Ladakh and the apricot trees were in full bloom. Two magpies had already begun their day and were busy looking for insects to eat. Her Ama-ley was awake too. She could hear her in the kitchen making gur-gur tea. Chuskit had been awake for the last hour. This was a day she was going to remember for a long time. Can you guess why? No, it was not Losar, the New Year festival. That was many months away. Nor was it any special day in her village, like the gonpa festival, or a wedding day. Today was going to be Chuskit's first day at school. At nine years, she had waited a long, long time for this. The school was not very far away from her home. To get there, you had to walk up to the main road. Just before the prayer wheel, you took the path to the left of the road that ran along a narrow stream. Near..."
4,"Not Now, Not Now!","I asked Ajji, “May I please have some laddoos?” “Not now, beta, maybe tomorrow.” But I did not want to wait till tomorrow. I asked Ajja, “May I go out to play with my friends?” “Not now, beta. Maybe after your nap?”But I did not want to wait till after my nap. I asked Ma, “May I wear these new clothes?” “Not now, beta, you can wear them when we go out.” But I did not want to wait till we went out. I asked Papa, “May I open that nice looking box?” “Not now, not now. You must wait.” But I did not want to wait any more. Why do big people always say, ‘Not now, not now’? That night, I went to sleep feeling angry. The next day, I woke up early and went into the kitchen. Ajji said, “You can eat these laddoos now.” Ajja said, “We can play cricket now.” Ma said, “You can wear these clothes now.” Papa said, “You can open this box now.” And they all said, “Happy birthday!”"


In [8]:
# Merge both summary and content. 
df_english_stories['summary_and_content'] = df_english_stories['stories_summary'] + " " + df_english_stories['story_content']

In [9]:
# There is one story with null content. Drop it.
df_english_stories[df_english_stories['summary_and_content'].isnull()]

,story_id,story_title,story_english_title,is_child_created_story,stories_status,stories_summary,ancestry,is_recommended_story,reads,language_name,organization_name,page_type,story_derivation_type,story_publishing_type,reading_level_cat,story_content,summary_and_content
1311,18480,A man with a cat,NaN,t,1,A man with a cat.,NaN,f,37,English,NaN,StoryPage,Original,UGC Story,1,NaN,NaN


In [10]:
df_english_stories.dropna(subset=['summary_and_content'], inplace=True)

In [11]:
df_english_stories.head(1)

,story_id,story_title,story_english_title,is_child_created_story,stories_status,stories_summary,ancestry,is_recommended_story,reads,language_name,organization_name,page_type,story_derivation_type,story_publishing_type,reading_level_cat,story_content,summary_and_content
0,2,Smile Please!,Smile Please!,f,1,Follow the young deer as he races along with friends.,1.0,t,13981,English,Pratham Books,StoryPage,Translated,Publisher Story,1,"A fawn was racing in the forest. He was ahead of the rabbit. He was ahead of the elephant. He leapt and cleared the stream. He ran past the crumbling wall. There was a large boulder on the grassy plain. He stumbled and fell down. He burst into tears. The monkey massaged his leg. Tears flowed from the fawn's eyes. Brother Bear picked him up. The fawn didn't stop crying. His mother came. She said, “Look, we’ll beat up this bad boulder!” The fawn said, “Oh, don’t do that or he will also start crying.” His mother laughed. So did the fawn.","Follow the young deer as he races along with friends. A fawn was racing in the forest. He was ahead of the rabbit. He was ahead of the elephant. He leapt and cleared the stream. He ran past the crumbling wall. There was a large boulder on the grassy plain. He stumbled and fell down. He burst into tears. The monkey massaged his leg. Tears flowed from the fawn's eyes. Brother Bear picked him up. The fawn didn't stop crying. His mother came. She said, “Look, we’ll beat up this bad boulder!” The fawn said, “Oh, don’t do that or he will also start crying.” His mother laughed. So did the fawn."


### Following are a set of utility functions which is used to clean the text

In [12]:
def strip_html_tags(text):
    '''
    Strip HTML tags
    '''
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

strip_html_tags('<html><h2>Some important text</h2></html>')

'Some important text'

In [13]:
# Remove accented characters
def remove_accented_chars(text):
    '''
    Remove accented characters
    '''
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

remove_accented_chars('Sómě Áccěntěd těxt')

'Some Accented text'

In [14]:
# Expand contractions
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())                       
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
        
    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

expand_contractions("Y'all can't expand contractions I'd think")

'You all cannot expand contractions I would think'

In [15]:
# Remove special characters
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    return text

remove_special_characters("Well this was fun! What do you think? 123#@!", 
                          remove_digits=True)

'Well this was fun What do you think '

In [16]:
# Text lemmatization
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

lemmatize_text("My system keeps crashing! his crashed yesterday, ours crashes daily")

'My system keep crash ! his crashed yesterday , ours crash daily'

In [17]:
# Text stemming
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

simple_stemmer("My system keeps crashing his crashed yesterday, ours crashes daily")

'My system keep crash hi crash yesterday, our crash daili'

In [18]:
# Remove Stop words
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

remove_stopwords("The, and, if are stopwords, computer is not")

', , stopwords , computer not'

In [19]:
def normalize_corpus(corpus, html_stripping=True, contraction_expansion=True,
                     accented_char_removal=True, text_lower_case=True, 
                     text_lemmatization=True, special_char_removal=True, 
                     stopword_removal=True, remove_digits=True):
    '''
    Utility function to pre-process the text
    '''
    try:
        doc = corpus
        # strip HTML
        if html_stripping:
            doc = strip_html_tags(doc)
        # remove accented characters
        if accented_char_removal:
            doc = remove_accented_chars(doc)
        # expand contractions    
        if contraction_expansion:
            doc = expand_contractions(doc)
        # lowercase the text    
        if text_lower_case:
            doc = doc.lower()
        # remove extra newlines
        doc = re.sub(r'[\r|\n|\r\n]+', ' ',doc)
        # lemmatize text
        if text_lemmatization:
            doc = lemmatize_text(doc)
        # remove special characters and\or digits    
        if special_char_removal:
            # insert spaces between special characters to isolate them    
            special_char_pattern = re.compile(r'([{.(-)!}])')
            doc = special_char_pattern.sub(" \\1 ", doc)
            doc = remove_special_characters(doc, remove_digits=remove_digits)  
        # remove extra whitespace
        doc = re.sub(' +', ' ', doc)
        # remove stopwords
        if stopword_removal:
            doc = remove_stopwords(doc, is_lower_case=text_lower_case)
    except:
        print(corpus)
        
    return doc

In [20]:
# Check the original content
df_english_stories.iloc[0]['summary_and_content']

"Follow the young deer as he races along with friends. A fawn was racing in the forest. He was ahead of the rabbit. He was ahead of the elephant. He leapt and cleared the stream. He ran past the crumbling wall. There was a large boulder on the grassy plain. He stumbled and fell down. He burst into tears. The monkey massaged his leg. Tears flowed from the fawn's eyes. Brother Bear picked him up. The fawn didn't stop crying. His mother came. She said, “Look, we’ll beat up this bad boulder!” The fawn said, “Oh, don’t do that or he will also start crying.” His mother laughed. So did the fawn."

In [21]:
# Check the normalized content
normalize_corpus(df_english_stories.iloc[0]['summary_and_content'])

'follow young deer race along friend fawn race forest ahead rabbit ahead elephant leap clear stream run past crumble wall large boulder grassy plain stumble fall burst tear monkey massage leg tear flow fawn eye brother bear pick fawn not stop cry mother come say look well beat bad boulder fawn say oh not also start cry mother laugh fawn'

In [22]:
# Generate the normalized/pre-processed content for all the stories
df_english_stories['preprocessed_story_content'] = df_english_stories.summary_and_content.apply(lambda x : normalize_corpus(x))

In [24]:
# Check the content the column clean_story_content
df_english_stories['preprocessed_story_content'][0]

'follow young deer race along friend fawn race forest ahead rabbit ahead elephant leap clear stream run past crumble wall large boulder grassy plain stumble fall burst tear monkey massage leg tear flow fawn eye brother bear pick fawn not stop cry mother come say look well beat bad boulder fawn say oh not also start cry mother laugh fawn'

In [25]:
# Remove the summary_and_content and content column. This may not be needed.
df_english_stories = df_english_stories.drop(labels=['summary_and_content'], axis='columns')

In [26]:
df_english_stories.to_csv("stories_pre_processed_content_english.csv", sep=",")